In [1]:
import thermoextrap.xtrapy.xpan_beta as xpan_beta
import xarray as xr
import numpy as np
import thermoextrap.xtrapy.core as xtrapy_core

In [58]:
def stack_xr_for_gp(da, xdims, ydims=None, 
                    xname='xstack',
                    yname='ystack',
                    variable=None,
                    policy='raise'):
    """
    Given an xarray.DataArray, stack into new shape for analysis
    
    Parameters
    ----------
    dims_x : str or tuple of strings
        dimensions to consider for X
    dims_y : str, tuple of strings, optional
        if passed, these dimensions are stacked.  If not passed,
        use all remaining dimensions
    variable : str or tuple of strings, optional
        use this to indicate a dimension that contains variables (mean and variance).
        This dimenension is moved to the last position.
    policy : {'raise', 'infer'}
        policy if coordinates not available
    
    Returns
    -------
    x : array, shape=(N, D)
        feature array to consider.  this will be in the order specified by dims_x
    y : array, sahpe=(N, P)
    
    
    """
    dims = da.dims
    for name in [xname, yname]:
        if name in dims:
            raise ValueError('{} conficts with existing {}'.format(xname, dims))
    
    if isinstance(xdims, str):
        xdims = (xdims,)
        
    stacker = {xname : xdims}
    if isinstance(ydims, str):
        ydims = (ydims,)
    elif ydims is None:
        ydims = tuple(set(dims) - set(xdims))
    
    if len(ydims) > 0:
        stacker[yname] = ydims
        
    if policy == 'raise':
        for dim in xdims:
            if dim not in da.coords:
                raise ValueError('da.coords[{}] not set'.format(dim))
    
    out = da.stack(**stacker)
    
    if variable is not None:
        if isinstance(variable, str):
            variable = (variable,)
        out = out.transpose(...,*variable)
    
    return out

def wrap_like(x, da):
    """
    wrap an array x with properties of da
    """
    return xr.DataArray(x, dims=da.dims, coords=da.coords, indexes=da.indexes, attrs=da.attrs, name=da.name)


def get_x_from_stack(stack, xname='xstack'):
    """
    get numpy array of X coordinates
    """
    da = stack[xname]
    out = np.array(list(da.values))
    return out



import pandas as pd
def to_ave_var(da, dim_reduce, dim_concat=None, concat_kws=None, reduce_kws=None):
    """
    for a dataarray apply average/variance along a dimension
    
    new array has dimension `dim_concat` which denotes mean and variance
    
    """
    if reduce_kws is None:
        reduce_kws = {}
    if concat_kws is None:
        concat_kws = {}
        
    if dim_concat is None:
        dim_concat = pd.Index(['mean','var'], name='variable')
    
    return xr.concat(
        (da.mean(dim_reduce, **reduce_kws), 
         da.var(dim_reduce, **reduce_kws)), 
        dim=dim_concat, **concat_kws)
    
    
    

In [ ]:
class WrapForGP(xtrapy_core.StateCollection):
    
    def __init__(self, states, 
                 xdims, 
                 ydim, 
                 xname, 
                 yname, 
                 ):
        
    
    
        
    

In [59]:
# synthetic data
# pretending data for rdf(pair, r)
# here, pair is for a-a, a-b, b-b
shape = (20, 3, 5)
dims = ['rec', 'pair', 'position']

coords = {'position': np.linspace(0, 2, shape[-1])}

xems = []
for beta in [0.1, 10.0]:
    x = xr.DataArray(np.random.rand(*shape), dims=dims, coords=coords)
    u = xr.DataArray(np.random.rand(shape[0]), dims=dims[0])
    data = xpan_beta.DataCentralMomentsVals.from_vals(x, u, order=3, central=True)
    xems.append(xpan_beta.factory_extrapmodel(beta, data))

s = xtrapy_core.StateCollection(xems)

In [60]:
# resample the object to take mean/var
sr = s.resample(nrep=100)
# create Y data to consider:
Y_unstack = xr.concat(
    (x.xcoefs(norm=False).pipe(to_ave_var, 'rep') for x in sr), 
    dim=pd.Index(sr.alpha0, name=sr.alpha_name))

In [64]:
Y = stack_xr_for_gp(Y_unstack, xdims=['beta','order'], ydims=['pair','position'], policy='infer', variable='variable')

In [66]:
X = get_x_from_stack(Y)

In [67]:
for meta, y in Y.groupby('ystack'):
    print('pair: {}, position: {}'.format(*meta))
    break

pair: 0, position: 0.0


In [77]:
Y.indexes['xstack'].names

FrozenList(['beta', 'order'])

In [ ]:
np.random.rand(100)

In [ ]:
def wrap_X_input_array():
    """provide an array x with same shape as target X"""
    pass